## Daily Electricity Demand (GWh) by U.S. Region
By: Aileen, Chengpu, and Aria

We imported the requests package and submitted a request to the EIA API to get the data using the url link. Aria had to have an API key sent to her email to add to the URL for authentication. We printed the json version to make sure it had imported correctly.

In [99]:
import requests
url = "https://api.eia.gov/v2/electricity/rto/daily-region-sub-ba-data/data/?frequency=daily&data[0]=value&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000&api_key=LTIKgNRTzB9PyNmYCHTfG8sr3A4h2E1RH6O9Swfi"
data_request = requests.get(url)
data = data_request.json()
print(data)


{'warnings': [{'warning': 'incomplete return', 'description': 'The API can only return 5000 rows in JSON format.  Please consider constraining your request with facet, start, or end, or using offset to paginate results.'}], 'response': {'total': '1045365', 'dateFormat': 'YYYY-MM-DD', 'frequency': 'daily', 'data': [{'period': '2026-02-17', 'subba': 'PGAE', 'subba-name': 'Pacific Gas and Electric', 'parent': 'CISO', 'parent-name': 'California Independent System Operator', 'timezone': 'Arizona', 'value': '285432', 'value-units': 'megawatthours'}, {'period': '2026-02-17', 'subba': 'PGAE', 'subba-name': 'Pacific Gas and Electric', 'parent': 'CISO', 'parent-name': 'California Independent System Operator', 'timezone': 'Central', 'value': '285105', 'value-units': 'megawatthours'}, {'period': '2026-02-17', 'subba': 'PGAE', 'subba-name': 'Pacific Gas and Electric', 'parent': 'CISO', 'parent-name': 'California Independent System Operator', 'timezone': 'Eastern', 'value': '284623', 'value-units': 

Noticed something was off when running json commands, so figured out it imported as type "dict."

In [100]:
import pandas as pd
type(data)


dict

Converted it to a normalized json. Then, I imported the data, which was nested inside the root key, "response," and within that, in a key "data." Then, I convered and printed the first few rows of the dataframe.

In [101]:

df = pd.json_normalize(data["response"]["data"])
df = pd.DataFrame(df)
type(df)
print(df.shape)
df.head()



(5000, 8)


,period,subba,subba-name,parent,parent-name,timezone,value,value-units
0,2026-02-17,PGAE,Pacific Gas and Electric,CISO,California Independent System Operator,Arizona,285432,megawatthours
1,2026-02-17,PGAE,Pacific Gas and Electric,CISO,California Independent System Operator,Central,285105,megawatthours
2,2026-02-17,PGAE,Pacific Gas and Electric,CISO,California Independent System Operator,Eastern,284623,megawatthours
3,2026-02-17,PGAE,Pacific Gas and Electric,CISO,California Independent System Operator,Mountain,285432,megawatthours
4,2026-02-17,PGAE,Pacific Gas and Electric,CISO,California Independent System Operator,Pacific,285907,megawatthours


Converted the demand float type and made a new column with the generation figure divided by 1000 so that the unit is gigawatt hours rather than megawatt hours (smaller numbers, easier to read).

In [102]:
df["value"] = df["value"].astype(float)
df["valueGWh"] = df["value"] / 1000


Here, we made a new data frame aggregating by date and region the total amount of generation. Now we have a data frame with three columns, date, region, and demand in gigawatts

In [103]:
new_df = (
    df.groupby(["period", "parent"])
    .agg(Demand=("valueGWh", "sum"))
    .reset_index()
)
print(new_df)
new_df.shape


         period parent    Demand
0    2026-02-03   CISO  2944.611
1    2026-02-03   ERCO  3800.828
2    2026-02-04   CISO  2970.943
3    2026-02-04   ERCO  6082.567
4    2026-02-04   ISNE  1901.063
..          ...    ...       ...
96   2026-02-17   CISO  3136.398
97   2026-02-17   MISO  1692.924
98   2026-02-17   NYIS   432.930
99   2026-02-17    PNM   159.490
100  2026-02-17   SWPP  1590.912

[101 rows x 3 columns]


(101, 3)

Did pretty much all the same steps so that we have a data frame with three columns: date, fuel type, and demand. This shows us, across the US, how much of each fuel type was consumed each day. This dataset also shows generation by balancing authority, or the responsible entity that maintains real-time, instantaneous balance between electricity supply (generation) and demand (load) within a designated grid area. If we want to combine these two datasets to make a visualization eventually, we will need to also find a key that matches balancing authority to grid region (aka parent or ISO).

In [104]:
url2 = "https://api.eia.gov/v2/electricity/rto/daily-fuel-type-data/data/?frequency=daily&data[0]=value&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000&api_key=LTIKgNRTzB9PyNmYCHTfG8sr3A4h2E1RH6O9Swfi"
data_request2 = requests.get(url2)
data2 = data_request2.json()
print(data2)


{'warnings': [{'warning': 'incomplete return', 'description': 'The API can only return 5000 rows in JSON format.  Please consider constraining your request with facet, start, or end, or using offset to paginate results.'}], 'response': {'total': '5159839', 'dateFormat': 'YYYY-MM-DD', 'frequency': 'daily', 'data': [{'period': '2026-02-17', 'respondent': 'AECI', 'respondent-name': 'Associated Electric Cooperative, Inc.', 'fueltype': 'COL', 'type-name': 'Coal', 'timezone': 'Arizona', 'timezone-description': 'Arizona', 'value': '35981', 'value-units': 'megawatthours'}, {'period': '2026-02-17', 'respondent': 'AECI', 'respondent-name': 'Associated Electric Cooperative, Inc.', 'fueltype': 'COL', 'type-name': 'Coal', 'timezone': 'Central', 'timezone-description': 'Central', 'value': '36625', 'value-units': 'megawatthours'}, {'period': '2026-02-17', 'respondent': 'AECI', 'respondent-name': 'Associated Electric Cooperative, Inc.', 'fueltype': 'COL', 'type-name': 'Coal', 'timezone': 'Eastern', 't

In [108]:
df2 = pd.json_normalize(data2["response"]["data"])
df2 = pd.DataFrame(df2)
type(df2)
print(df2.shape)
df2.head()


(5000, 9)


,period,respondent,respondent-name,fueltype,type-name,timezone,timezone-description,value,value-units
0,2026-02-17,AECI,"Associated Electric Cooperative, Inc.",COL,Coal,Arizona,Arizona,35981,megawatthours
1,2026-02-17,AECI,"Associated Electric Cooperative, Inc.",COL,Coal,Central,Central,36625,megawatthours
2,2026-02-17,AECI,"Associated Electric Cooperative, Inc.",COL,Coal,Eastern,Eastern,37275,megawatthours
3,2026-02-17,AECI,"Associated Electric Cooperative, Inc.",COL,Coal,Mountain,Mountain,35981,megawatthours
4,2026-02-17,AECI,"Associated Electric Cooperative, Inc.",COL,Coal,Pacific,Pacific,35981,megawatthours


In [109]:
df2["value"] = df["value"].astype(float)
df2["valueGWh"] = df["value"] / 1000
new_df2 = df2.groupby(["period", "type-name"]).agg(Demand=("value", "sum")).reset_index()
print(new_df2)
new_df2.shape


        period                              type-name      Demand
0   2026-02-15                        Battery storage   1084607.0
1   2026-02-15                                   Coal   1722265.0
2   2026-02-15                             Geothermal    211196.0
3   2026-02-15                                  Hydro   6722076.0
4   2026-02-15                            Natural Gas   7327060.0
5   2026-02-15                                Nuclear   2511926.0
6   2026-02-15                                  Other   6258203.0
7   2026-02-15                              Petroleum   2175876.0
8   2026-02-15                         Pumped storage   1940575.0
9   2026-02-15                                  Solar   6947930.0
10  2026-02-15  Solar with integrated battery storage   2197324.0
11  2026-02-15                 Unknown energy storage    844531.0
12  2026-02-15                                   Wind   5644084.0
13  2026-02-16                        Battery storage  15870022.0
14  2026-0

(45, 3)